In [1]:
import lightgbm as lgb
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score
import joblib
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('winequality-red.csv',delimiter=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [7]:
#возможные градации качества вина
df.value_counts("quality")

quality
5    681
6    638
7    199
4     53
8     18
3     10
dtype: int64

In [8]:
X = df.drop('quality', axis = 1)
y = df['quality']

In [9]:

# Преобразование метки класса так, чтобы они начинались с 0
y_encoded, y_labels = pd.factorize(y)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Создание и обучение модели LightGBM
d_train = lgb.Dataset(X_train, label=y_train)
params = {
    'objective': 'multiclass',
    'num_class': 4, # Обновлено в соответствии с количеством уникальных классов
    'metric': 'multi_logloss',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}
model = lgb.train(params, d_train, num_boost_round=100)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Обратное преобразование предсказанных метк
y_pred_original = y_labels.inverse_transform(y_pred)

# Оценка модели
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

# Присвоение значений метрик переменным для мониторинга
f1_score_value = f1
precision_score_value = precision
recall_score_value = recall

# Вывод значений метрик
print(f"F1 Score: {f1_score_value}")
print(f"Precision: {precision_score_value}")
print(f"Recall: {recall_score_value}")

# Вывод предсказанных метк в оригинальном формате
# print("Предсказанные метки в оригинальном формате:")
# print(y_pred_original)



LightGBMError: Label must be in [0, 6), but found 6 in label